# Exploring the Output

In the [former tutorial](./dglap.ipynb) we computed an evolution operator, but we didn't use it.
In this one, we'll then explore the content a structure of a computed operator.

In [1]:
import eko

First of all, we need to recompute the operator, since we didn't save it.

In [2]:
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
evolution_operator = eko.run_dglap(th_card, op_card)

The computed operator is now it's own type (that no longer inherits from dictionary as in previous eko versions):

In [3]:
type(evolution_operator)

eko.output.struct.EKO

Now, let's inspect the content of the operator: e.g. you can extract the theory and operator card

In [4]:
# obtain theory card
print(evolution_operator.theory_card)
# or operator card
print(evolution_operator.operator_card)

{'CKM': '0.97428 0.22530 0.003470 0.22520 0.97345 0.041000 0.00862 0.04030 0.999152', 'Comments': 'LO baseline for small-x res', 'DAMP': 0, 'EScaleVar': 1, 'FNS': 'FFNS', 'GF': 1.1663787e-05, 'HQ': 'POLE', 'IB': 0, 'IC': 0, 'ID': 0, 'MP': 0.938, 'MW': 80.398, 'MZ': 91.1876, 'MaxNfAs': 6, 'MaxNfPdf': 6, 'ModEv': 'EXA', 'ModSV': None, 'NfFF': 3, 'PTO': 0, 'Q0': 1.0, 'QED': 0, 'Qedref': 1.777, 'Qmb': 4.5, 'Qmc': 2.0, 'Qmt': 173.07, 'Qref': 91.2, 'SIN2TW': 0.23126, 'SxOrd': 'LL', 'SxRes': 0, 'TMC': 0, 'XIF': 1.0, 'XIR': 1.0, 'alphaqed': 0.007496251999999999, 'alphas': 0.11800000000000001, 'fact_to_ren_scale_ratio': 1.0, 'global_nx': 0, 'kDISbThr': 1.0, 'kDIScThr': 1.0, 'kDIStThr': 1.0, 'kbThr': 1.0, 'kcThr': 1.0, 'ktThr': 1.0, 'mb': 4.5, 'mc': 2.0, 'mt': 173.07, 'nf0': None, 'nfref': None}
{'Q0': 1.0, 'Q2grid': [100], 'backward_inversion': 'expanded', 'configs': {'backward_inversion': 'expanded', 'ev_op_iterations': 10, 'ev_op_max_order': [10, 0], 'interpolation_is_log': True, 'interpolati

So an `Output` object has some internal parameters, related to the interpolation used for the calculation, and then some external attributes, related to the final operator delivered.
But actually, we have not accessed yet the actual operator - let's first find out again which final scales we computed:

In [5]:
evolution_operator.Q2grid

array([100])

Remember that the unique starting scale is $Q_0^2$. In the present case there is a unique final scale, but in the general one there might be many.
Now, let's use this operator! The recommended way to load an operator is by using a context manager:

In [6]:
with evolution_operator.operator(100) as op:
    print(f"operator: {op.operator.shape}")
    print(f"error: {op.error.shape}")

operator: (14, 5, 14, 5)
error: (14, 5, 14, 5)


This is the final product we expected from the beginning: the evolution operator, delivered as a numerical array.
It is actually composed by two elements:

- the **operator** itself,  whose dimensions are `(flavor_out, x_out, flavor_in, x_in)`
- the *error* on each operator element, propagated from the integration error on the numerical Mellin inversion (no other source is taken into account)

How to use this object is now completely up to the user, but a few helpers are included in another package: `ekobox`!

This package will be explored in [a separate tutorial](./pdf.ipynb).